<a href="https://colab.research.google.com/github/jkranyak/birdscompetitionkaggle/blob/main/notebooka3489e63aa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'birdclef-2024:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F70203%2F8068726%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240411%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240411T234502Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D46e0e22533c4fcd389c01d0e4a59680f0195c48981d89b38fbcc56169788c4e2ec0cd598173d4b060798187fc098d9ae8be44f928e19332fd7a04dd9291b7cd5812e6ccb7a55cde8fb478dd218fbb7838d1668800383c690cadf71fdb848aef8fc877fe66aa901c88f53b367b5159910d89ce41d7d6e5efaa398ab4edb3a3698eb791cf66685b93ce0b06ccbe72381b605d5a1cd8666e4a5b0c3deed470d2e40bcebf7459e89a08001c6c206049a51e26d8448f234521076fa17a96a87092b56e9f57708917dce91638fcc740d304c9522406f9230270c574117522f32a1760132f49b58bdb9ccae232fd02fe940370249bd14d548dc193e0be81f80f0badebf'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[============                                      ] 5897707520 bytes downloaded

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install librosa soundfile tensorflow scikit-learn numpy pandas matplotlib seaborn tqdm
!pip install audiomentations
!pip install tqdm


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, LSTM, TimeDistributed, Dense, Dropout, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from keras.utils import Sequence
from keras.preprocessing.image import load_img, img_to_array

import os
import librosa
import librosa.display
import soundfile as sf
import torch
import torch.nn as nn
from tqdm import tqdm

import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
import multiprocessing as mp

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

import joblib
from multiprocessing import Pool
from scipy import signal
from scipy.io import wavfile
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift


In [ ]:
import os
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
from tqdm import tqdm

# Helper function to construct the full path to the audio file
def create_audio_path(row, base_audio_dir):
    return os.path.join(base_audio_dir, row['primary_label'], row['filename'])

# Function to perform audio augmentation
def advanced_audio_augmentation(audio_signal, sr):
    time_stretch_factor = np.random.uniform(0.9, 1.1)
    audio_signal = librosa.effects.time_stretch(audio_signal, rate=time_stretch_factor)
    n_steps = np.random.randint(-2, 3)
    audio_signal = librosa.effects.pitch_shift(audio_signal, sr, n_steps=n_steps)
    noise_level = np.random.uniform(0.001, 0.005)
    noise = np.random.normal(0, noise_level, len(audio_signal))
    audio_signal += noise
    return audio_signal

# Function to create and save a spectrogram
def create_and_save_spectrogram(audio_path, save_path):
    try:
        y, sr = librosa.load(audio_path, sr=None)
        y_augmented = advanced_audio_augmentation(y, sr)
        S = librosa.feature.melspectrogram(y=y_augmented, sr=sr)
        S_DB = librosa.power_to_db(S, ref=np.max)
        plt.figure(figsize=(10, 4))
        librosa.display.specshow(S_DB, sr=sr, x_axis='time', y_axis='mel')
        plt.colorbar(format='%+2.0f dB')
        plt.title('Mel-frequency spectrogram')
        plt.tight_layout()
        plt.savefig(save_path)
        plt.close()
    except Exception as e:
        print(f"Failed to process {audio_path}: {str(e)}")

# Process all audio files
def process_audio_files(metadata, base_audio_dir, output_dir):
    print(f"Processing {len(metadata)} files...")
    for _, row in tqdm(metadata.iterrows(), total=len(metadata)):
        audio_path = create_audio_path(row, base_audio_dir)
        save_path = os.path.join(output_dir, f"{os.path.splitext(row['filename'])[0]}_spectrogram.png")
        if os.path.exists(audio_path):
            create_and_save_spectrogram(audio_path, save_path)
        else:
            print(f"File not found: {audio_path}")

# Main code
metadata_path = '/kaggle/input/birdclef-2024/train_metadata.csv'
metadata = pd.read_csv(metadata_path)
filtered_metadata = metadata[metadata['latitude'].between(8, 21) & metadata['longitude'].between(72, 78)]

base_audio_dir = '/kaggle/input/birdclef-2024/train_audio'
output_dir = '/kaggle/working/processed_data'
os.makedirs(output_dir, exist_ok=True)

# Set the audio path correctly
filtered_metadata['audio_path'] = filtered_metadata.apply(lambda row: create_audio_path(row, base_audio_dir), axis=1)

# Process the audio files
process_audio_files(filtered_metadata, base_audio_dir, output_dir)
